In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('final file hopefully no errors.csv')
df['Len'] = [len(s) for s in df['Seq'].to_list()]

dfp = df.loc[df['Gen'] == 'ssRNA(+)']
dfn = df.loc[df['Gen'] == 'ssRNA(-)']

wdfp = dfp.sample(len(dfn))

wdfp = wdfp.reset_index(drop = True)
dfp = dfp.reset_index(drop = True)
dfn = dfn.reset_index(drop = True)

In [4]:
#taking all ATGC permutations for windows of length 7

import itertools

winlist7 = []

for i in range(7,8):

    for i in (itertools.product('ATGC', repeat=i)):
        winlist7.append(''.join(i))

In [7]:
#load databases as dataframes

#set fragment/window size

window_sizes = list(range(7,8))

#set count threshold
###

#set other parameters
match_award      = 2
mismatch_penalty = -1
gap_penalty      = -1 # both for opening and extanding

#complete workflow

for win in window_sizes:
    winlist = [w for w in winlist7 if len(w) == win]
    
    #taking all unique elements into winlist
    
#     for repseq in repseqs:
#         for i in range(0, len(repseq) - win + 1):
#             piece = repseq[i:i + win]
#             if piece not in winlist:
#                 winlist.append(piece)
                
    print('Done making %d windows' % len(winlist)) 
    
    #two dataframes to stores results
    
    resultp = pd.DataFrame(index = list(range(len(dfp))), columns = winlist)
    resultn = pd.DataFrame(index = list(range(len(dfn))), columns = winlist)
    
    c = 1
    
    for element in winlist:
        for i, r in dfp.iterrows():
            score = calculate_element_score3(match_award, mismatch_penalty, gap_penalty, element, r['Seq'])
            
            resultp.loc[i, element] = score
            
        for i, r in dfn.iterrows():
            score = calculate_element_score3(match_award, mismatch_penalty, gap_penalty, element, r['Seq'])
            
            resultn.loc[i, element] = score
            
        print('%d iterations done in winlist. %d remaining' % (c, (len(winlist) - c) ))
        
        c+=1

    resultp.to_csv('resultp window %d.csv' % win, index = False)
    resultn.to_csv('resultn window %d.csv' % win, index = False)

Done making 16384 windows


NameError: name 'calculate_element_score3' is not defined

In [8]:
def calculate_element_score3(match_award, mismatch_penalty, gap_penalty, seq1, seq2):
    m, n = len(seq1), len(seq2)  # length of two sequences

    # Generate DP table and traceback path pointer matrix
    score = np.zeros((m+1, n+1)).astype(int)      # the DP table
                                                    # to store the traceback path

    max_score = 0        # initial maximum score in DP table
    # Calculate DP table and mark pointers

    scores_list = []

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            score_diagonal = score[i-1][j-1] + match_score(seq1[i-1], seq2[j-1])
            score_up = score[i][j-1] + gap_penalty
            score_left = score[i-1][j] + gap_penalty
            score[i][j] = max(0,score_left, score_up, score_diagonal)
            if score[i][j] >= max_score:
                max_i = i
                max_j = j
                max_score = score[i][j];

    perfect_score = min(len(seq1), len(seq2)) * match_award

    perfect_matches = np.count_nonzero(score == perfect_score)

    first_tolerance_match = np.count_nonzero(score == perfect_score - 1)
    second_tolerance_match = np.count_nonzero(score == perfect_score - 2)
    third_tolerance_match = np.count_nonzero(score == perfect_score - 3)

    final_score = perfect_matches + first_tolerance_match/(perfect_score * 2) + second_tolerance_match/(perfect_score * 4) + third_tolerance_match/(perfect_score * 8)
    
    return final_score

def match_score(alpha, beta):
    if alpha == beta:
        return match_award
    elif alpha == '-' or beta == '-':
        return gap_penalty
    else:
        return mismatch_penalty